In [ ]:
# Package Imports
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import sklearn
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

# Load Data and Create Sparse Matrix

In [ ]:
# Read books data
books = pd.read_csv('red_books.csv')
books

In [ ]:
# Read sparse matrix encoding into df
sparse_df = pd.read_csv('users_sparse.csv')
sparse_df

In [ ]:
# Create Scipy sparse matrix and convert to compressed sparse row format for operations
row = np.array(sparse_df.r_index)
col = np.array(sparse_df.c_index)
dat = np.array(sparse_df.data)
users = coo_matrix((dat,(row,col)),shape=(np.unique(row).size,np.unique(col).size))
users = users.tocsr()

# Determining k

In [ ]:
n_users = np.unique(row).size
n_users

In [ ]:
# From some source somewhere
k = (math.sqrt(n_users))/2
k = round(k)
k

In [ ]:
# Systematic approach

# KNN for Cosine Similarities

In [ ]:
# Set up KNN
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=377)
knn.fit(users)

# Pick a user id from 0 to 567805
user_id = 478

# Get k neighbors for user_id
nbrs = knn.kneighbors(users.getrow(user_id))

# Gives user index and distance (angle between vectors, 0 means cosine simlarity == 1)
nbrs

# With Cosine Similarity Function Instead

In [ ]:
# Pick a user id from 0 to 567805
user_id = 478

# Calculate pairwise cosine similarities between user and all other users
similarities = cosine_similarity(users.getrow(user_id),users)

# Get top k similar users that are not equal to 1
sim_index = pd.DataFrame({'sim':similarities.flatten()})
sim_index = sim_index.sort_values(['sim'])
sim_index = sim_index[sim_index['sim'] != 1]
nbrs = sim_index.iloc[-k:]
nbrs = nbrs.reset_index()

# Weighted Average

In [ ]:
# Books read by the user_id
user_books = np.array(sparse_df[sparse_df['r_index']==user_id].c_index)

# Total similarity
total_sim = sum(nbrs.sim)

# Loop through each neighbor and find set difference and total the set differences
set_diff = np.empty(0,dtype=int)

for ind in nbrs['index']:
    nbr_books = np.array(sparse_df[sparse_df['r_index']==ind].c_index)
    set_diff = np.union1d(set_diff,np.setdiff1d(nbr_books,user_books))

In [ ]:
# Get rows from users cooresponding to nbrs
nbrs_books = users[nbrs.index]
# Get columns cooresponding to books read by neighbors but not user
nbrs_books = nbrs_books[:,set_diff]

# Convert to dataframe for ease of computation
nbrs_books = pd.DataFrame.sparse.from_spmatrix(nbrs_books)
nbrs_books